In [ ]:
from wordcloud import WordCloud
import imageio
import sys, os
import numpy as np
import requests
import json
from flask import Flask, jsonify, render_template, request, redirect, url_for
from flask import abort
from bs4 import BeautifulSoup


from textblob import TextBlob
from textblob import Word
from textblob.sentiments import NaiveBayesAnalyzer
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from operator import itemgetter
import pandas as pd
import matplotlib.pyplot as plt
import imageio
from wordcloud import WordCloud
from PIL import Image
import re, datetime
import shutil
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

In [3]:
GENIUS_API_KEY = 'iPRpGvyPEPwHqexfQo75LsL0i2pPQxhkw-P5WStYbdvmUq-PQyf7ppCnT92Z-ZQc'
genius_base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer ' + GENIUS_API_KEY}

In [4]:
nrc_df = pd.read_csv("static/data/NRC-Emotion-Lexicon-v0.92.csv")

In [5]:
nrc_df

,Word,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,1,0,0,0,1,0,1,0,0
3,abandoned,0,1,1,0,0,1,0,1,0,0
4,abandonment,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
14177,zone,0,0,0,0,0,0,0,0,0,0
14178,zoo,0,0,0,0,0,0,0,0,0,0
14179,zoological,0,0,0,0,0,0,0,0,0,0
14180,zoology,0,0,0,0,0,0,0,0,0,0


In [6]:
def get_song_details(song_id):
    song_dict = {}

    # Get song metadata
    search_url = genius_base_url + '/songs/' + str(song_id)
    response = requests.get(search_url, headers=headers)
    song = response.json()
    song_dict['id'] = song['response']['song']['id']
    song_dict['title'] = song['response']['song']['title']
    song_dict['artist'] = song['response']['song']['primary_artist']['name']
    song_dict['artist_image'] = song['response']['song']['primary_artist']['image_url']
    song_dict['url'] = song['response']['song']['url']
    song_dict['album'] = song['response']['song']['album']['name']
    song_dict['year'] = song['response']['song']['release_date']
    song_dict['song_image'] = song['response']['song']['song_art_image_thumbnail_url']

    # Get song lyrics
    response = requests.get(song['response']['song']['url'], headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    lyrics = soup.find('div', class_='lyrics').get_text()
    lyrics = lyrics.replace('\n', ' ').replace('\r', ' ')
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    song_dict['lyrics'] = lyrics
    # print(song_dict)
    return song_dict

In [7]:
def preprocess(lyrics_input):
    lyrics = str(lyrics_input)
    lyrics = lyrics.lower()
    lyrics = lyrics.replace('{html}', "")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', lyrics)
    rem_url = re.sub(r'http\S+', '', cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if len( w) > 2 if not w in stopwords.words('english')]
    return " ".join(filtered_words)

In [8]:
song_data_dict = get_song_details(2336647)

In [9]:
lyrics = song_data_dict['lyrics']
lyrics = lyrics.lower()
lyrics

"   i will leave my heart at the door i won't say a word they've all been said before, you know so why don't we just play pretend like we're not scared of what is coming next or scared of having nothing left?   look, don't get me wrong i know there is no tomorrow all i ask is   if this is my last night with you hold me like i'm more than just a friend give me a memory i can use take me by the hand while we do what lovers do it matters how this ends 'cause what if i never love again?   i don't need your honesty it's already in your eyes and i'm sure my eyes, they speak for me no one knows me like you do and since you're the only one that mattered tell me, who do i run to?   look, don't get me wrong i know there is no tomorrow all i ask is   if this is my last night with you hold me like i'm more than just a friend give me a memory i can use take me by the hand while we do what lovers do it matters how this ends 'cause what if i never love again?   let this be our lesson in love let this

In [10]:
lyrics = preprocess(lyrics)

In [ ]:
tokens=RegexpTokenizer(r'\w+').tokenize(lyrics)
tokens

In [12]:
def build_nrc_sentiment(lyrics_input):
    tokens=RegexpTokenizer(r'\w+').tokenize(lyrics_input)
    lyrics_df=pd.DataFrame(tokens, columns=['Word'])
    lyrics_df=lyrics_df.merge(nrc_df, how="inner", on="Word")
    lyrics_df=lyrics_df.drop(columns=["Word"])
    results_df=pd.DataFrame(lyrics_df.sum())
    return results_df.to_dict()[0]

In [16]:
build_nrc_sentiment(lyrics)

{'Positive': 10,
 'Negative': 6,
 'Anger': 2,
 'Anticipation': 4,
 'Disgust': 2,
 'Fear': 1,
 'Joy': 7,
 'Sadness': 2,
 'Surprise': 1,
 'Trust': 6}

In [19]:
lyrics_df=pd.DataFrame(tokens, columns=['Word'])

In [20]:
lyrics_df = lyrics_df.merge(nrc_df, how="inner", on="Word")
lyrics_df = lyrics_df.drop(columns=["Word"])

In [54]:
results_df= pd.DataFrame(lyrics_df.sum(),columns=['Weight'])

In [60]:
results_df.sort_values(by=['Weight'], ascending=False, inplace=True)

In [61]:
results_df

,Weight
Positive,10
Joy,7
Negative,6
Trust,6
Anticipation,4
Anger,2
Disgust,2
Sadness,2
Fear,1
Surprise,1


In [63]:
list(results_df.to_dict()['Weight'].keys())

['Positive',
 'Joy',
 'Negative',
 'Trust',
 'Anticipation',
 'Anger',
 'Disgust',
 'Sadness',
 'Fear',
 'Surprise']

In [64]:
results_df.to_dict()['Weight'].values()

dict_values([10, 7, 6, 6, 4, 2, 2, 2, 1, 1])

In [66]:
final = {}
final['Sentiment'] = list(results_df.to_dict()['Weight'].keys())
final['Weight'] = list(results_df.to_dict()['Weight'].values())

In [67]:
final

{'Sentiment': ['Positive',
  'Joy',
  'Negative',
  'Trust',
  'Anticipation',
  'Anger',
  'Disgust',
  'Sadness',
  'Fear',
  'Surprise'],
 'Weight': [10, 7, 6, 6, 4, 2, 2, 2, 1, 1]}